In [4]:
from shared import getQdrantClient, getEmbeddingsModel
qClient = getQdrantClient()

# Show everything in the Document collection
numDocumentChunks = 0
# Note with_vectors defaults to false, so the vectors are not returned
chunks = qClient.scroll(collection_name='Document', limit=100)
while True:
    for chunk in chunks[0]:
        if numDocumentChunks == 0:
            sampleDocumentChunk = chunk
        numDocumentChunks += 1
    chunks = qClient.scroll(collection_name='Document', limit=100, with_payload=False, offset=chunks[1])
    if chunks[1] is None:
        break
print("Number of document chunks: ", numDocumentChunks)
if numDocumentChunks > 0:
    print("\nSample document chunk(metadata not the vector): ")
    print(sampleDocumentChunk, '\n')

# Show everything in the Github collection
numGithubChunks = 0
# Note with_vectors defaults to false, so the vectors are not returned
chunks = qClient.scroll(collection_name='Github', limit=100)
while True:
    for chunk in chunks[0]:
        if numGithubChunks == 0:
            sampleGithubChunk = chunk
        numGithubChunks += 1
    chunks = qClient.scroll(collection_name='Github', limit=100, with_payload=False, offset=chunks[1])
    if chunks[1] is None:
        break
print("Number of githb chunks: ", numDocumentChunks)
if numGithubChunks > 0:
    print("\nSample github chunk(metadata not the vector): ")
    print(sampleGithubChunk, '\n')

# Show a sample search
embeddingsModel = getEmbeddingsModel()
results = qClient.search(
    collection_name="Document",
    query_vector = embeddingsModel.embed_query("What operating system is ROS made for?"),
    limit=10
)
print("\nSample search result(n=2): ")
for result in results:
    print(result)

Number of document chunks:  0
Number of githb chunks:  0

Sample search result(n=2): 


In [22]:
import numpy as np
# How cosine distance works

embedding1 = embeddingsModel.embed_query("What is the weather like?")
embedding2 = embeddingsModel.embed_query("It is raining today.")
embedding3 = embeddingsModel.embed_query("ROS is an open source platform")
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)
similarity1 = cosine_similarity(embedding1, embedding2)
similarity2 = cosine_similarity(embedding1, embedding3)
print("Cosine Similarity for related sentences:", similarity1)
print("Cosine Similarity for unrelated sentences:", similarity2)

Cosine Similarity for related sentences: 0.7035977848391597
Cosine Similarity for unrelated sentences: 0.3566534327076298


In [3]:
from qdrant_client.http.models import Distance, VectorParams
# Delete all collections and vectors inside them
qClient.delete_collection(collection_name = "Document")
qClient.delete_collection(collection_name = "Github")
# Recreate the empty collections
qClient.create_collection(
    collection_name = "Document",
    vectors_config=VectorParams(size=3072, distance=Distance.COSINE)
)
qClient.create_collection(
    collection_name = "Github",
    vectors_config=VectorParams(size=3072, distance=Distance.COSINE)
)

True